In [19]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from random import randint
from time import sleep
import time

In [20]:
review = pd.read_csv('../data/year_2018.csv')

In [21]:
review.drop('Unnamed: 0',axis = 1, inplace = True)

In [22]:
review.shape

(377430, 9)

In [23]:
review.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewText,summary
0,5.0,NaN,True,2018-10-04,A2IXGBWKX73O21,B01HI7D4VY,NaN,"quality, very bright","for the price, excellent"
1,5.0,NaN,True,2018-10-04,AHEX3FYTJ2AX6,B01HH36F74,{'Color:': ' CA100'},Works as expected.,Works as expected.
2,3.0,NaN,False,2018-10-03,A2J77LP43D7QMX,B01HF0YGCK,{'Color:': ' Black'},For a 3 pack they had a great price when I bou...,Great for the price
3,5.0,NaN,True,2018-10-03,A32FN9RNY65CKH,B01HGF6XTI,{'Size:': ' Charger'},Went to Boston and forgot my charger.\nOrdered...,Worked fine - got me through in a pinch
4,5.0,NaN,True,2018-10-02,A2ZE26LV7HVSAD,B01GUITK24,{'Style Name:': ' Bookshelf Speaker Mount 2 Pa...,I had to use my own drywall anchors which were...,Solid and kind of heavy


In [24]:
asin_list = list(review['asin'].unique())

In [25]:
len(asin_list)

56675

In [26]:
len(list(review['reviewerID'].unique()))

168153

In [17]:
driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")
product_list = []
driver.get('https://www.amazon.com/gp/product/B01GUITK24')
soup = BeautifulSoup(driver.page_source, 'lxml')
product_info = {}
ul_d = soup.find('ul', {'class': 'a-unordered-list a-vertical a-spacing-none'})
product_info['description'] = [litag.text.strip() for litag in ul_d.find_all('li')]
product_info

AttributeError: 'NoneType' object has no attribute 'find_all'

In [33]:
#chrome_options = Options()
#chrome_options.add_argument('--dns-prefetch-disable')
#webdriver= Chrome(chrome_options = chrome_options)
driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")
product_list = []
counter = 0
t1=time.time()
for i in asin_list[14001:15000]:
    try:
        driver.get(f'https://www.amazon.com/gp/product/{i}')
    except:
        driver.close()
        driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")
        driver.get(f'https://www.amazon.com/gp/product/{i}')
    soup = BeautifulSoup(driver.page_source, 'lxml')
    product_info = {}
    try:
        product_info['asin'] = i
        product_info['product_name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        product_info['product_brand'] = soup.find('a', {'id':'bylineInfo'}).text.strip()
        product_info['product_rating'] = soup.find('span', {'class':'a-icon-alt'}).text.strip()
        product_info['product_rate_count'] = soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        ul_c= soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'})
        product_info['category'] = [litag.text.strip() for litag in ul_c.find_all('li') if litag.text.strip() != '›']
        ul_d = soup.find('ul', {'class': 'a-unordered-list a-vertical a-spacing-none'})
        product_info['description'] = [litag.text.strip() for litag in ul_d.find_all('li')]
        product_list.append(product_info)
    except:
        product_info['asin'] = i
        product_info['product_name'] = 0
        product_info['product_brand'] = 0
        product_info['product_rating'] = 0
        product_info['product_rate_count'] = 0
        product_info['category'] = 0
        product_info['description'] = 0
        
        product_list.append(product_info)
    counter+=1
    if counter%100 == 0:
        print (counter,time.time()-t1)
    sleep(randint(1,2))
product = pd.DataFrame(product_list)

100 467.5344271659851
200 933.9704563617706
300 1430.7179863452911
400 1866.4886393547058
500 2263.1523926258087
600 2668.629253387451
700 3059.070291042328
800 3469.301524400711
900 3907.997594833374


In [29]:
product

,asin,product_name,product_brand,product_rating,product_rate_count,category,description
0,B00HZDRCRO,0,0,0,0,0,0
1,B00RAXEM9K,SoundPal SonoBass Wireless Over-Ear Headphones...,SoundPal,3.6 out of 5 stars,225 ratings,"[Electronics, Headphones, Over-Ear Headphones]",[Consumer Alert: Most users do not need a lice...
2,B01BLZ8BA8,TV Remote Add-on for Fire TV Stick Remote (Con...,Mission Cables,3.4 out of 5 stars,308 ratings,"[Electronics, Accessories & Supplies, Audio & ...",[Control your TV directly from your Fire TV St...
3,B00Q7V8NR8,iPad Air 2 A1567 Slim Tough Case G5 - Rugged P...,Hitek,3.4 out of 5 stars,13 ratings,"[Electronics, Computers & Accessories, Tablet ...",[Super rugged protection for the iPad Air 2 A1...
4,B00KCVZ2FA,"STK EN-EL15 EN-EL15a Battery for Nikon D7000, ...",STK/SterlingTek,4.2 out of 5 stars,"1,004 ratings","[Electronics, Camera & Photo, Accessories, Bat...",[Make sure this fits\nby entering your model n...
...,...,...,...,...,...,...,...
994,B00LGN8OW6,"Kingwin 4-Port USB 3.0 Hub w/LED for MacBook, ...",Kingwin,3.2 out of 5 stars,158 ratings,"[Electronics, Computers & Accessories, Network...",[Make sure this fits\nby entering your model n...
995,B017D9K6A4,0,0,0,0,0,0
996,B00F5UDW2I,Kattee Men's Canvas Cow Leather DSLR SLR Vinta...,Kattee,4.4 out of 5 stars,164 ratings,"[Electronics, Camera & Photo, Bags & Cases, Ca...",[Make sure this fits\nby entering your model n...
997,B00W962GJQ,Bestcompu Genuine New Lenovo Thinkpad L540 W54...,Best Compu,4.1 out of 5 stars,9 ratings,"[Electronics, Computers & Accessories, Compute...",[Make sure this fits\nby entering your model n...


In [34]:
product.to_csv('../data/product14001_15000.csv')